In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar  6 15:55:40 2022

@author: 99259
"""
import tensorflow as tf
import package_dataprocess.Nilm_classes
import numpy as np
import keras
#from nilm_ukdale import preprocess
from keras import Model
from keras.models import Sequential
import os
import gc
import pandas as pd
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import PIL.ImageOps
from keras import Model
from keras import layers
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import MaxPooling3D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import GRU,TimeDistributed,Bidirectional
from keras.layers import Embedding
import matplotlib.pyplot as plt

一切都是命运石之门的选择！


In [2]:
#电器id定义
fridge_id = 0
tv_id = 1
kettle_id = 2
microwave_id = 3
washerdryer_id = 4

#电器开启状态阈值
fridge_threshold = 20
tv_threshold = 20
kettle_threshold = 20
microwave_threshold = 10
washerdryer_threshold = 0

dp = package_dataprocess.Nilm_classes.Nilm_dataprocess(path_csv="DATA/UKData_by_hour/week1/vicurve_name.xlsx",path_img="DATA/UKData_by_hour/week1/vicurve32x32/")
sum_data_num = 0

filenames=os.listdir(r'DATA/UKData_2/')
print(filenames)




['Active subwoofer0.npy', 'Audio amplifier0.npy', 'Audio system0.npy', 'Baby monitor0.npy', 'Boiler0.npy', 'Breadmaker0.npy', 'Broadband router0.npy', 'Charger0.npy', 'Clothes iron0.npy', 'Coffee maker0.npy', 'Computer monitor0.npy', 'Computer0.npy', 'Desktop computer0.npy', 'Dish washer0.npy', 'Ethernet switch0.npy', 'Fan0.npy', 'Fridge freezer0.npy', 'Hair dryer0.npy', 'Hair straighteners0.npy', 'HTPC0.npy', 'Kettle0.npy', 'Laptop computer0.npy', 'Light0.npy', 'Light1.npy', 'Light10.npy', 'Light11.npy', 'Light12.npy', 'Light13.npy', 'Light14.npy', 'Light2.npy', 'Light3.npy', 'Light4.npy', 'Light5.npy', 'Light6.npy', 'Light7.npy', 'Light8.npy', 'Light9.npy', 'Microwave0.npy', 'Mobile phone charger0.npy', 'Mobile phone charger1.npy', 'Oven0.npy', 'Printer0.npy', 'Radio0.npy', 'Radio1.npy', 'Solar thermal pumping station0.npy', 'Soldering iron0.npy', 'Tablet computer charger0.npy', 'Television0.npy', 'Toaster0.npy', 'Vacuum cleaner0.npy', 'Washer dryer0.npy', 'Wireless phone charger0.np

In [3]:
for i in filenames:
    
    name = i[:-4]
    name_data = np.load('DATA/UKData_2/'+i)
    if len(name_data)>=100800:
        name_data = name_data[:100800]
        name_data = np.where(name_data > 10,name_data,0)
        sum_data_num = sum_data_num + name_data
    else:
        print(i[:-4])

Clothes iron0
Hair straighteners0


In [4]:
fridge_data = np.load('DATA/UKData_2/Fridge freezer0.npy')
fridge_data = fridge_data[:100800]
fridge_data = np.where(fridge_data > dp.air_threshold,fridge_data,0)

television_data = np.load('DATA/UKData_2/Television0.npy')
television_data = television_data[:100800]
television_data = np.where(television_data > dp.air_threshold, television_data,0)

kettle_data = np.load('DATA/UKData_2/Kettle0.npy')
kettle_data = kettle_data[:100800]
kettle_data = np.where(kettle_data > dp.air_threshold, kettle_data,0)

microwave_data = np.load('DATA/UKData_2/Microwave0.npy')
microwave_data = microwave_data[:100800]
microwave_data = np.where(microwave_data > dp.air_threshold, microwave_data,0)

washerdryer_data = np.load('DATA/UKData_2/Washer dryer0.npy')
washerdryer_data = washerdryer_data[:100800]
washerdryer_data = np.where(washerdryer_data > dp.air_threshold, washerdryer_data,0)

Fan_data = np.load('DATA/UKData_2/Fan0.npy')
Fan_data = fridge_data[:100800]
Fan_data = np.where(fridge_data > dp.air_threshold,Fan_data,0)



In [5]:
print(fridge_data.shape)

(100800, 1)


In [5]:
fridge_labels = dp.create_label(fridge_data,1,fridge_threshold)
tv_labels = dp.create_label(television_data,4,tv_threshold)
kettle_labels = dp.create_label(kettle_data,2,kettle_threshold)
microwave_labels = dp.create_label(microwave_data,3,microwave_threshold)
washerdryer_labels = dp.create_label(washerdryer_data,5,washerdryer_threshold)
Fan_labels = dp.create_label(Fan_data,0,5)

sum_label = fridge_labels + tv_labels + kettle_labels + microwave_labels + washerdryer_labels+Fan_labels

mean = sum_data_num[:100800].mean(axis=0)
sum_data = sum_data_num - mean
std = sum_data[:100800].std(axis=0)
sum_data /= std


In [6]:
print(sum_data.shape)

(100800, 1)


In [6]:
train_gen = dp.generator_mix(
                      data = sum_data,
                      label=sum_label,
                      min_index=0,
                      max_index=85000)

val_gen = dp.generator_mix(
                    label=sum_label,
                    data = sum_data,
                    min_index=85001,
                    max_index=95000)
test_gen = dp.generator_mix(
                    data = sum_data,
                    label=sum_label,
                    min_index=95001,
                    max_index=100800)

train_steps = 85000 // dp.batch_size
val_steps = (95000 - 85001 -dp.lookback) // dp.batch_size
test_steps = (100800 - 95001 -dp.lookback) // dp.batch_size


In [7]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
from keras.layers import InputLayer

In [8]:
def create_cnn():
    model = Sequential()
    model.add(InputLayer(input_shape=(None,32,32,1)))
    model.add(TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu')))
    #model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),strides=(2,2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),strides=(2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(TimeDistributed(Dense(64, activation='relu')))
    model.add(layers.Bidirectional(layers.GRU(64, dropout=0.2)))
    model.add(layers.Dense(16,activation='relu'))
    #model.add(TimeDistributed(Embedding(input_dim=2222,output_dim=32)))
    #model.add(layers.Dense(32,activation='relu'))
    # return our model
    return model

In [9]:
def create_seq():
    model = Sequential()
    model.add(InputLayer(input_shape=(None,5)))
    model.add(layers.Bidirectional(
            layers.GRU(32, dropout=0.2)))
    model.add(layers.Dense(16,activation='relu'))
    return model

In [10]:
branch_cnn = create_cnn()
branch_seq = create_seq()
combined = concatenate([branch_seq.output, branch_cnn.output])
x = Dense(32,activation='relu')(combined)
#x = TimeDistributed(Dense(64, activation='relu'))(x)
#x = layers.GRU(32,dropout=0.2)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=[branch_seq.input, branch_cnn.input], outputs=x)

In [11]:
filepath='CNNRNNMODEL.hdf5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath=filepath,
                                   verbose=1, monitor='val_loss', save_weights_only=False, mode = 'max')

model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])

In [12]:

history = model.fit_generator(train_gen,
                              steps_per_epoch=train_steps,
                              epochs=4,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[checkpointer])
model.summary()

E:\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/4
   9/1328 [..............................] - ETA: 1:07:11 - loss: 0.5765 - acc: 0.3924

KeyboardInterrupt: 

In [13]:
model.save("my_h5_model.h5")

In [8]:
mymodel=tf.keras.models.load_model('CNNRNNMODEL.hdf5')

In [9]:
test_loss, test_acc = mymodel.evaluate_generator(test_gen,steps=test_steps)
print('test acc : ', test_acc)

E:\anaconda3\lib\site-packages\keras\engine\training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


KeyboardInterrupt: 

#keras.callbacks.ModelCheckpoint('G:\model', monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=5)
model = Sequential()

model.add(TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu',input_shape=(None,32,32,1))))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),strides=(2,2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),strides=(2,2))))
model.add(TimeDistributed(Flatten()))
#model.add(TimeDistributed(Dense(64, activation='relu')))
#model.add(TimeDistributed(Embedding(input_dim=2222,output_dim=32)))
model.add(layers.Bidirectional(
    layers.GRU(64,dropout=0.1)))
model.add(layers.Dense(5,activation='sigmoid'))
#model.build(input_shape=(None,32,32,1))
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])
#model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['acc'])
history = model.fit_generator(train_gen,
                              steps_per_epoch=train_steps,
                              epochs=7,
                              validation_data=val_gen,
                              validation_steps=val_steps)
model.summary()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(loss)+1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
# plt.legend()
plt.show()

plt.clf() #清空图表

acc_values = history.history['acc']
val_acc_values  = history.history['val_acc']

plt.plot(epochs,acc_values,'bo',label='Training acc') #bo是蓝色圆点
plt.plot(epochs,val_acc_values,'b',label='Validation acc') #b是蓝色实线
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

test_loss, test_acc = model.evaluate_generator(test_gen,steps=test_steps)
print('test acc : ', test_acc)